# Model Registry Setup

## Purpose
This notebook creates a SageMaker Model Package Group for organizing and versioning flight delay prediction models. The model package group serves as a container for all versions of models registered in the SageMaker Model Registry.

## Prerequisites
None - this notebook handles all setup requirements.

## Output
- Model Package Group: `flight-delay-models`
- The notebook is idempotent and safe to re-run

In [1]:
import boto3
import sagemaker
from botocore.exceptions import ClientError

print(f"SageMaker SDK Version: {sagemaker.__version__}")
print(f"AWS Region: {boto3.Session().region_name}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
SageMaker SDK Version: 2.245.0
AWS Region: us-east-1


In [2]:
sm_client = boto3.client('sagemaker')
model_package_group_name = 'flight-delay-models'

print(f"Model Package Group Name: {model_package_group_name}")

Model Package Group Name: flight-delay-models


In [4]:
try:
    response = sm_client.describe_model_package_group(
        ModelPackageGroupName=model_package_group_name
    )
    print(f"Model package group '{model_package_group_name}' already exists.")
    print(f"ARN: {response['ModelPackageGroupArn']}")
    print(f"Status: {response['ModelPackageGroupStatus']}")
    
except ClientError as e:
    error_code = e.response['Error']['Code']
    if error_code in ['ValidationException', 'ResourceNotFound']:
        print(f"Creating model package group '{model_package_group_name}'...")
        response = sm_client.create_model_package_group(
            ModelPackageGroupName=model_package_group_name,
            ModelPackageGroupDescription="Flight delay prediction models for AAI540 group project"
        )
        print(f"Model package group created successfully.")
        print(f"ARN: {response['ModelPackageGroupArn']}")
    else:
        raise

Creating model package group 'flight-delay-models'...
Model package group created successfully.
ARN: arn:aws:sagemaker:us-east-1:786869526001:model-package-group/flight-delay-models


In [5]:
print("Setup complete!")
print(f"\nModel Package Group: {model_package_group_name}")
print(f"Region: {boto3.Session().region_name}")
print("\nYou can now register models to this group using:")
print(f"  model_package_group_name = '{model_package_group_name}'")

Setup complete!

Model Package Group: flight-delay-models
Region: us-east-1

You can now register models to this group using:
  model_package_group_name = 'flight-delay-models'


## Model Registry Result

The `flight-delay-models` model package group is now ready to receive model registrations. In our training notebooks:

1. **Train your model** using SageMaker training jobs or local experiments
2. **Register the model** to this group using the SageMaker SDK:
   ```python
   model.register(
       model_package_group_name='flight-delay-models',
       approval_status='PendingManualApproval',
       inference_instances=['ml.t2.medium'],
       transform_instances=['ml.m5.large']
   )
   ```
3. **Review and approve** models through the SageMaker console or SDK
4. **Deploy approved models** to endpoints for real-time or batch inference

This setup enables systematic model versioning and deployment governance for your flight delay prediction project.